# Preprocess

In [13]:
import pandas as pd

In [14]:
df = pd.read_csv("/content/US-Economic-News.csv", delimiter=',', encoding= 'ISO-8859-1')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _unit_id               8000 non-null   int64  
 1   _golden                8000 non-null   bool   
 2   _unit_state            8000 non-null   object 
 3   _trusted_judgments     8000 non-null   int64  
 4   _last_judgment_at      8000 non-null   object 
 5   positivity             1420 non-null   float64
 6   positivity:confidence  3775 non-null   float64
 7   relevance              8000 non-null   object 
 8   relevance:confidence   8000 non-null   float64
 9   articleid              8000 non-null   object 
 10  date                   8000 non-null   object 
 11  headline               8000 non-null   object 
 12  positivity_gold        0 non-null      float64
 13  relevance_gold         0 non-null      float64
 14  text                   8000 non-null   object 
dtypes: b

In [15]:
df.head(5)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
0,842613455,False,finalized,3,12/5/15 17:48,3.0,0.6400,yes,0.640,wsj_398217788,8/14/91,Yields on CDs Fell in the Latest Week,NaN,NaN,NEW YORK -- Yields on most certificates of dep...
1,842613456,False,finalized,3,12/5/15 16:54,NaN,NaN,no,1.000,wsj_399019502,8/21/07,The Morning Brief: White House Seeks to Limit ...,NaN,NaN,The Wall Street Journal Online</br></br>The Mo...
2,842613457,False,finalized,3,12/5/15 1:59,NaN,NaN,no,1.000,wsj_398284048,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,NaN,NaN,WASHINGTON -- In an effort to achieve banking ...
3,842613458,False,finalized,3,12/5/15 2:19,NaN,0.0000,no,0.675,wsj_397959018,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,NaN,NaN,The statistics on the enormous costs of employ...
4,842613459,False,finalized,3,12/5/15 17:48,3.0,0.3257,yes,0.640,wsj_398838054,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NaN,NaN,NEW YORK -- Indecision marked the dollar's ton...


In [16]:
# tengo que agarrar y cambiar los dtype de las variables con encoding
# drop positivity gold and relevance gold
# positivity and ::confidence, have very sparse values
# _unit_state has only the finalized observation and golden the False observation, and trusted judgments only observation '3'

df = df.drop(columns = ['_unit_id', 'positivity_gold', 'relevance_gold', '_unit_state', '_golden', '_trusted_judgments',
                         '_unit_state', '_last_judgment_at', 'positivity', 'positivity:confidence', 'relevance:confidence', 'articleid',
                       'date'], axis = 1)

#df['articleid'] = df['articleid'].str.slice(0, 3)
# so far, these variables SHOULD be dropped for sure as well as unit id and article id
# should discuss between the positivity and positivity confidence due to amount of null values

In [17]:
df.head(5)

,relevance,headline,text
0,yes,Yields on CDs Fell in the Latest Week,NEW YORK -- Yields on most certificates of dep...
1,no,The Morning Brief: White House Seeks to Limit ...,The Wall Street Journal Online</br></br>The Mo...
2,no,Banking Bill Negotiators Set Compromise --- Pl...,WASHINGTON -- In an effort to achieve banking ...
3,no,Manager's Journal: Sniffing Out Drug Abusers I...,The statistics on the enormous costs of employ...
4,yes,Currency Trading: Dollar Remains in Tight Rang...,NEW YORK -- Indecision marked the dollar's ton...


Cleaning Strings

In [18]:
#!pip install nltk

In [19]:
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [27]:
# #Ensure you have downloaded the necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [21]:
df['whole_txt'] = df['headline']+ ' ' + df['text']

In [22]:
wtxt_train = np.array(df['whole_txt'])


In [23]:
print(wtxt_train)

['Yields on CDs Fell in the Latest Week NEW YORK -- Yields on most certificates of deposit offered by major banks dropped more than a tenth of a percentage point in the latest week, reflecting the overall decline in short-term interest rates.</br></br>On small-denomination, or "consumer," CDs sold directly by banks, the average yield on six-month deposits fell to 5.49% from 5.62% in the week ended yesterday, according to an 18-bank survey by Banxquote Money Markets, a Wilmington, Del., information service.</br></br>On three-month "consumer" deposits, the average yield sank to 5.29% from 5.42% the week before, according to Banxquote. Two banks in the Banxquote survey, Citibank in New York and CoreStates in Pennsylvania, are paying less than 5% on threemonth small-denomination CDs.</br></br>Declines were somewhat smaller on five-year consumer CDs, which eased to 7.37% from 7.45%, Banxquote said.</br></br>Yields on three-month and six-month Treasury bills sold at Monday\'s auction plummet

In [24]:
for i in range(len(wtxt_train)):
    # Deletion of non-latin alfabet signs, also numbers
    wtxt_train[i] = re.sub(r'[^a-zA-Z]', ' ', wtxt_train[i])
    # Removing single letter works like 'a'.
    wtxt_train[i] = re.sub(r"\s+[a-zA-Z]\s+", ' ', wtxt_train[i])
    # Removing double spaces
    wtxt_train[i] = re.sub(r'\s+', ' ', wtxt_train[i])
    # Lower case
    wtxt_train[i] = wtxt_train[i].lower()

Tokenization

In [28]:
for i in range(len(wtxt_train)):
    wtxt_train[i] = word_tokenize(wtxt_train[i])

Removing stop words

In [29]:
stop_words = set(stopwords.words('english'))

for i in range(len(wtxt_train)):
    wtxt_train[i] = [word for word in wtxt_train[i] if word not in stop_words]

In [ ]:
wtxt_train[0]
# stop_words

Lemmatization

In [31]:
lemmatizer = WordNetLemmatizer()
for i in range(len(wtxt_train)):
    wtxt_train[i] = [lemmatizer.lemmatize(word) for word in wtxt_train[i]]

In [32]:
df['whole_txt'] = wtxt_train
df = df.drop(['headline', 'text'], axis = 1)

In [33]:
df.head(5)

,relevance,whole_txt
0,yes,"[yield, cd, fell, latest, week, new, york, yie..."
1,no,"[morning, brief, white, house, seek, limit, ch..."
2,no,"[banking, bill, negotiator, set, compromise, p..."
3,no,"[manager, journal, sniffing, drug, abuser, qui..."
4,yes,"[currency, trading, dollar, remains, tight, ra..."


In [34]:
## Importing Libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

### Data preparation
* Initial Data Processing: Our first step is to encode the relevance label into both the Relevant (1) and non-Relevant labels (0). Then, we make it into a np.array to feed into the model.
* Then, we begin to clean text data into pad sequences.

In [101]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline

y = pd.DataFrame(df, columns=['relevance'])

# Define the transformation: OneHotEncoder for categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_columns)
    ],
    remainder='passthrough'  # This will pass through other columns not specified in transformers
)

# Create a pipeline with the preprocessor
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Fit and transform the DataFrame
y_ = pipeline.fit_transform(y)

y_ = pd.DataFrame(y_, columns=pipeline.named_steps['preprocessor'].get_feature_names_out())
y_ = y_.astype(int)

In [102]:
y_ = y_.drop(columns=['cat__relevance_not sure'])

In [114]:
labels = np.array(y_)
labels # export to make models

array([[0, 1],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]])

### Tokenization
First, we need to "tokenize" our sentences, i.e., convert them to sequences of numbers. For this task, we are going to use the `Tokenizer` from Tensorflow (documentation [here](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer))

In [44]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(wtxt_train)   # fit our tokenizer on the dataset (i.e., assign a number to each word and keep a
                                    # dictionary with the correspondence of each word to a number)

# see the language dictionary and the total number of words (please note that number 0 is reserved for the padding task)
word_index = tokenizer.word_index
total_words = len(word_index) + 1

In [ ]:
word_index

In [38]:
total_words

36555

### Padding Sequences
Sentences and sequences tend to have different lengths, however our model is expecting equally sized observations.
Here we want to convert our texts to sequences and make them of the same length (in general, the lenght of the longest of our sequences). We are going to use here `pad_sequences` from Tensorflow (documentation [here](https://www.tensorflow.org/api_docs/python/tf/keras/utils/pad_sequences)), to add zeroes to the tokenized sentences until they all reach the same length.

In [41]:
sequences = tokenizer.texts_to_sequences(wtxt_train)
padded_sequences = pad_sequences(sequences)

In [ ]:
sequences

In [66]:
padded_sequences

array([[   0,    0,    0, ...,  382,   20, 2616],
       [   0,    0,    0, ...,  158,   50,  179],
       [   0,    0,    0, ...,   11,   84,   44],
       ...,
       [   0,    0,    0, ...,  515,  528,  121],
       [   0,    0,    0, ...,  279,  104,   60],
       [   0,    0,    0, ...,   42,  185,   23]], dtype=int32)

In [122]:
from sklearn.model_selection import train_test_split
train, test, labels_train, labels_test = train_test_split(wtxt_train, labels,  test_size=0.2, random_state=42, stratify=y)

### Building the model

We are going to build a simple model that includes:
- `Embedding` layer with an output representation of each word as a vector of dim 16
- `LSTM` (see class slides for more detail or RNNs example notebook for more details) with an intermediate state of 100
- An output layer `Dense` that connects the output of the LSTM and creates an output of 3 positions (one per class) as output of the network

In [84]:
embedding_dim = 16   # The output of our embedding layer. This is a hyperparameter that accounts for how long the
                     # vector representing each word is going to be

# We are going to build our model with the Sequential API
model = Sequential()
model.add(Embedding(total_words,      # number of words to process as input
                    100,    # output representation
                    input_length=len(padded_sequences[0])))    # total length of each observation
model.add(LSTM(100, return_sequences=False))
model.add(Dense(2, activation='sigmoid'))  # Change activation based on the number of classes

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [85]:
model(padded_sequences)

<tf.Tensor: shape=(8000, 2), dtype=float32, numpy=
array([[0.50148815, 0.5000838 ],
       [0.5009381 , 0.4995941 ],
       [0.4988029 , 0.5002627 ],
       ...,
       [0.499239  , 0.4999639 ],
       [0.50052494, 0.4987951 ],
       [0.49915776, 0.50016236]], dtype=float32)>

### Training the model

In [117]:
model.fit(train, labels_train, epochs=10)

Epoch 1/10
200/200 [==============================] - 73s 359ms/step - loss: 0.0095 - accuracy: 0.9975
Epoch 2/10
200/200 [==============================] - 72s 359ms/step - loss: 0.0067 - accuracy: 0.9981
Epoch 3/10
200/200 [==============================] - 70s 348ms/step - loss: 0.0049 - accuracy: 0.9981
Epoch 4/10
200/200 [==============================] - 71s 354ms/step - loss: 0.0058 - accuracy: 0.9984
Epoch 5/10
200/200 [==============================] - 71s 354ms/step - loss: 0.0049 - accuracy: 0.9981
Epoch 6/10
200/200 [==============================] - 70s 352ms/step - loss: 0.0068 - accuracy: 0.9980
Epoch 7/10
200/200 [==============================] - 70s 351ms/step - loss: 0.0164 - accuracy: 0.9931
Epoch 8/10
200/200 [==============================] - 72s 363ms/step - loss: 0.0075 - accuracy: 0.9980
Epoch 9/10
200/200 [==============================] - 71s 353ms/step - loss: 0.0071 - accuracy: 0.9973
Epoch 10/10
200/200 [==============================] - 71s 355ms/step - l

In [118]:
print(model(padded_sequences).numpy().argmax(axis = 1))

[1 0 0 ... 0 0 0]


Adjust the model architecture and hyperparameters based on your specific problem and data.

This is a basic example to get you started with implementing an RNN for text classification using TensorFlow. Fine-tune and expand upon this foundation according to your project requirements.

## Implementing an RNN for Text Classification with TensorFlow

In this example, we'll build a simple Recurrent Neural Network (RNN) using TensorFlow for text classification. The dataset consists of one or two sentences as input data.

This is a simplified and applied version with focus on the usage of Tensorflow. For a more detailed and extensive process description please refer to the "RNNs example" notebook or to the class presentations
